In [1]:
import mbuild as mb
import numpy as np

/Users/stephaniemccallum/miniforge3/envs/flowermd/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
class CY5(mb.Compound):
    def __init__(self):
        super(CY5, self).__init__(name="CY5")
        self.add(mb.load("CY5_backbone.mol2"))
        self.attachment_sites = {
            "left": {"nitrogen": self[22], "hydrogen": self[23]},
            "right": {"nitrogen": self[14], "hydrogen": self[35]}
        }
        self.capping_groups = {}

    def attachment_vector(self, site):
        head = self.attachment_sites[site]["hydrogen"]
        tail = self.attachment_sites[site]["nitrogen"]
        vec = head.pos - tail.pos
        return vec / np.linalg.norm(vec)

    def get_fixed_particles(self, site):
        group = self.capping_groups[site]
        return [p for p in group.particles()]
    
    def add_attachment_chain(self, site, chain_length, bond_length=0.145):
        hydrogen = self.attachment_sites[site]["hydrogen"]
        nitrogen = self.attachment_sites[site]["nitrogen"]
        last_bonding_atom = nitrogen
        for i in range(chain_length):
            if i != chain_length - 1:
                new_group = mb.lib.moieties.CH2()
            else:
                new_group = mb.lib.moieties.CH3()
                self.capping_groups[site] = new_group
            new_group.translate_to(nitrogen.pos)
            new_group.translate(by=self.attachment_vector(site) * bond_length * (i + 1))
            self.add(new_group)
            self.add_bond([new_group[0], last_bonding_atom])
            last_bonding_atom = new_group[0]
        self.remove(hydrogen)

In [3]:
cy5_backbone = CY5()
cy5_backbone.add_attachment_chain(site="left", chain_length=5)
cy5_backbone.add_attachment_chain(site="right", chain_length=5)
cy5_backbone.visualize()

/Users/stephaniemccallum/miniforge3/envs/flowermd/lib/python3.12/site-packages/gmso/formats/mol2.py:80: UserWarning: The record type indicator Meta is not supported. Skipping current section and moving to the next RTI header.
  warnings.warn(
/Users/stephaniemccallum/miniforge3/envs/flowermd/lib/python3.12/site-packages/gmso/formats/mol2.py:80: UserWarning: The record type indicator @<TRIPOS>SUBSTRUCTURE is not supported. Skipping current section and moving to the next RTI header.
  warnings.warn(
/Users/stephaniemccallum/miniforge3/envs/flowermd/lib/python3.12/site-packages/mbuild/compound.py:546: UserWarning: Some particle of <CY5 71 particles, 74 bonds, non-periodic, id: 6278445680> does not have a charge.They will not be accounted for during this calculation.
  warn(


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
cy5_backbone.get_fixed_particles("left")

[<C pos=([-0.5692  0.5055 -0.477 ]), 4 bonds, id: 6278521456>,
 <H pos=([-0.4622  0.5055 -0.477 ]), 1 bonds, id: 6278521888>,
 <H pos=([-0.6049  0.5824 -0.4117]), 1 bonds, id: 6278519920>,
 <H pos=([-0.6049  0.5236 -0.5763]), 1 bonds, id: 6278518576>]

## Building System and Applying Forcefield in flowerMD

In [5]:
from flowermd.base import Molecule, System, Pack
import gsd.hoomd
import hoomd

In [8]:
system = Pack(molecules=[cy5_backbone], density=0.5, packing_expand_factor=5)

In [9]:
from flowermd.library import GAFF

system.apply_forcefield(
    r_cut=2.5, force_field=GAFF(), auto_scale=True, scale_charges=True
)

ParameterizationError: Combining rules of the provided forcefields do notmatch, please provide forcefields with same scalingfactors that apply to a Topology